# seqFISH Mouse Organogenesis Imputed Extended

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>)
- **Date of Creation:** 01.08.2024
- **Date of Last Modification:** 12.08.2024

## 1. Setup

Run this notebook in the nichecompass-reproducibility environment, installable from ```('../../../envs/environment.yaml')```.

### 1.1 Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../utils")

In [ ]:
import math
import os
import warnings

import anndata as ad
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.sparse as sp
import seaborn as sns
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
import squidpy as sq
from nichecompass.benchmarking import compute_benchmarking_metrics
from nichecompass.models import NicheCompass
from nichecompass.utils import generate_enriched_gp_info_plots
from nichecompass.utils import create_new_color_dict

from analysis_utils import plot_category_in_latent_and_physical_space

In [ ]:
species = "mouse"
counts_key = "counts"
cat_covariates_keys = "batch"
gp_names_key = "nichecompass_gp_names"
differential_gp_test_results_key = "nichecompass_differential_gp_test_results"
spatial_key = "spatial"
adj_key = "spatial_connectivities"
n_neighbors = 8
latent_key = "nichecompass_latent"
sample_key = "sample"
mapping_entity_key = "mapping_entity"
spot_size = 0.05 

datasets = ["seqfish_mouse_organogenesis_imputed",
            "seqfish_mouse_organogenesis_imputed",
            "seqfish_mouse_organogenesis_imputed",
            "seqfish_mouse_organogenesis_imputed",
            "seqfish_mouse_organogenesis_imputed",
            "seqfish_mouse_organogenesis_imputed",
            "seqfish_mouse_organogenesis_imputed",
            "seqfish_mouse_organogenesis_imputed",
            "seqfish_mouse_organogenesis_imputed_subsample_50pct",
            "seqfish_mouse_organogenesis_imputed_subsample_25pct",
            "seqfish_mouse_organogenesis_imputed_subsample_10pct",
            "seqfish_mouse_organogenesis_imputed_subsample_5pct",
            "seqfish_mouse_organogenesis_imputed_subsample_1pct"]
load_timestamps = ["01082024_191836_3", # embryo 1,2 (reference) and embryo 3 (query)
                   "01082024_191835_2", # embryo 1,3 (reference and embryo 2 (query)
                   "02082024_111323_6", # all embryos (reference) and random seed 42
                   "02082024_111324_6", # all embryos (reference) and random seed 43
                   "02082024_002349_5", # all embryos (reference) and KNN 12
                   "02082024_002350_4", # all embryos (reference) and KNN 4
                   "07082024_124745_8", # embryo 2 (reference) and embryo 1,3 (query)
                   "07082024_124745_7", # embryo 1,2 (reference) and embryo 3 (query)
                   "08082024_095147_12", # embryo 2 50% (reference)
                   "08082024_095147_13", # embryo 2 25% (reference)
                   "08082024_140042_14", # embryo 2 10% (reference)
                   "08082024_140042_15", # embryo 2 5% (reference)
                   "08082024_140042_16", # embryo 2 1% (reference)
                   "07082024_105223_13", # NicheNet GPs de novo
                   "07082024_105223_15", # CollecTRI GPs de novo
                   "07082024_105223_12", # Omnipath GPs de novo
                   "07082024_105225_14", # MEBOCOST GPs de novo
                   "12082024_135236_17", # radius-based neighborhood graph
                  ]
batches = ["batch1", "batch2", "batch3", "batch4", "batch5", "batch6"]

### 1.2 Define Parameters

### 1.3 Run Notebook Setup

In [ ]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1.4 Configure Paths and Create Directories

In [ ]:
# Define paths
ga_data_folder_path = "../../datasets/ga_data"
gp_data_folder_path = "../../datasets/gp_data"
st_data_folder_path = "../../datasets/st_data"
st_data_gold_folder_path = st_data_folder_path + "/gold"
omnipath_lr_network_file_path = f"{gp_data_folder_path}/omnipath_lr_network.csv"
nichenet_lr_network_file_path = f"{gp_data_folder_path}/nichenet_lr_network_v2_{species}.csv"
nichenet_ligand_target_matrix_file_path = f"{gp_data_folder_path}/nichenet_ligand_target_matrix_v2_{species}.csv"
mebocost_enzyme_sensor_interactions_folder_path = f"{gp_data_folder_path}/metabolite_enzyme_sensor_gps"
gene_orthologs_mapping_file_path = f"{ga_data_folder_path}/human_mouse_gene_orthologs.csv"
artifacts_folder_path = f"../../artifacts"

## 2. Analysis

### 2.1. Embryo 2 Analysis with Leave Out Embryo 3

In [ ]:
model_label = "reference_query"
model_folder_path = f"{artifacts_folder_path}/{datasets[0]}/models/{model_label}/{load_timestamps[0]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[0]}/figures/{model_label}/{load_timestamps[0]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[0]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.25
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "3"
midbrain_cluster = "4"
hindbrain_cluster = "11"
floorplate_cluster = "19"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "10"
dorsalgut_cluster = "20"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gdf10 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf3 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.2. Embryo 2 Analysis with Leave Out Embryo 2

In [ ]:
model_label = "reference_query"
model_folder_path = f"{artifacts_folder_path}/{datasets[1]}/models/{model_label}/{load_timestamps[1]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[1]}/figures/{model_label}/{load_timestamps[1]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[1]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.3
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "3"
midbrain_cluster = "2"
hindbrain_cluster = "7"
floorplate_cluster = "20"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "1"
dorsalgut_cluster = "23"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gdf10 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf3 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.3. All Emybros Random Seed 42

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{datasets[2]}/models/{model_label}/{load_timestamps[2]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[2]}/figures/{model_label}/{load_timestamps[2]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[2]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.32
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "4"
midbrain_cluster = "6"
hindbrain_cluster = "3"
floorplate_cluster = "16"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "5"
dorsalgut_cluster = "19"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gdf10 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 0],
    show=False)
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Fgf3 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 1],
#    show=False) # inactive GP

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.4. All Emybros Random Seed 43

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{datasets[3]}/models/{model_label}/{load_timestamps[3]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[3]}/figures/{model_label}/{load_timestamps[3]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[3]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.32
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "5"
midbrain_cluster = "4"
hindbrain_cluster = "7"
floorplate_cluster = "17"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "1"
dorsalgut_cluster = "20"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Gdf10 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 0],
#    show=False) # inactive GP
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Fgf3 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 1],
#    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.5. All Emybros KNN 12

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{datasets[4]}/models/{model_label}/{load_timestamps[4]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[4]}/figures/{model_label}/{load_timestamps[4]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[4]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.35
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

sc.tl.leiden(adata=model.adata,
             resolution=0.05,
             key_added=latent_cluster_key,
             restrict_to=(latent_cluster_key, ["0"]),
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "3"
midbrain_cluster = "0,1"
hindbrain_cluster = "8"
floorplate_cluster = "15"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "2"
dorsalgut_cluster = "22"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Gdf10 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 0],
#    show=False) # inactive GP
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf3 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.6. All Emybros KNN 4

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{datasets[5]}/models/{model_label}/{load_timestamps[5]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[5]}/figures/{model_label}/{load_timestamps[5]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[5]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.35
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "6"
midbrain_cluster = "4"
hindbrain_cluster = "8"
floorplate_cluster = "16"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "5"
dorsalgut_cluster = "21"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Gdf10 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 0],
#    show=False) # inactive GP
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf3 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")

### 2.7. Embryo 3 Analysis with Leave Out Embryo 1 & 3

In [ ]:
model_label = "reference_query"
model_folder_path = f"{artifacts_folder_path}/{datasets[6]}/models/{model_label}/{load_timestamps[6]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[6]}/figures/{model_label}/{load_timestamps[6]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[6]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Plot mapping entities in latent and physical space
mapping_entity_colors = create_new_color_dict(
    adata=model.adata,
    skip_default_colors=63,
    cat_key=mapping_entity_key)

save_fig = True
file_path = f"{figure_folder_path}/" \
            "mapping_entities_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Mapping Entities",
    cat_key=mapping_entity_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=mapping_entity_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.25
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
presomitic_mesoderm_cluster = "19"

model.adata.obs[latent_cluster_key] = model.adata.obs[latent_cluster_key].apply(
    lambda x: "Presomitic Mesoderm" if x == "19" else x)

latent_cluster_colors["Presomitic Mesoderm"] = "#FF4D4D"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "presomitic_mesoderm_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=["Presomitic Mesoderm"],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=presomitic_mesoderm_cluster,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1

In [ ]:
enriched_gps

### 2.8 Embryo 3 Analysis with Leave Out Embryo 3

In [ ]:
model_label = "reference_query"
model_folder_path = f"{artifacts_folder_path}/{datasets[7]}/models/{model_label}/{load_timestamps[7]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[7]}/figures/{model_label}/{load_timestamps[7]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[7]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Plot mapping entities in latent and physical space
mapping_entity_colors = create_new_color_dict(
    adata=model.adata,
    skip_default_colors=63,
    cat_key=mapping_entity_key)

save_fig = True
file_path = f"{figure_folder_path}/" \
            "mapping_entities_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Mapping Entities",
    cat_key=mapping_entity_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=mapping_entity_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.25
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
presomitic_mesoderm_cluster = "18"

latent_cluster_colors[presomitic_mesoderm_cluster] = "#FF4D4D"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_cluster_18_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=["18"],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=presomitic_mesoderm_cluster,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1

In [ ]:
enriched_gps

In [ ]:
color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 20))

sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo3"],
    color=f"FGF18_ligand_receptor_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf18 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo3"],
    color=f"Dll1_ligand_receptor_target_gene_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dll1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo3"],
    color=f"Fgf18",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf18",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo3"],
    color=f"Dll1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dll1",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/presomitic_mesoderm_enriched.svg",
            bbox_inches="tight")

### 2.9 Embryo 3 Power Analysis

In [ ]:
model_label = "reference_query"
metrics = ["gcs", "mlami", "cas", "clisis", "nasw", "cnmi", "clisi", "blisi", "pcr"]

# Load niche labels
adata = sc.read_h5ad(f"{artifacts_folder_path}/seqfish_mouse_organogenesis_imputed/results/reference/09082023_225056_10/seqfish_mouse_organogenesis_imputed_analysis.h5ad")
niche_df = adata.obs[["niche"]]
    
# Initialize metrics dicts
metrics_dict_all = {"dataset": []}
metrics_dict_all["n_ref_cells"] = []
metrics_dict_all["nmi"] = []
for metric in metrics:
    metrics_dict_all[metric] = []

for idx in range(6, 13):
    metrics_dict_all["dataset"].append(datasets[idx])
    model_folder_path = f"{artifacts_folder_path}/{datasets[idx]}/models/{model_label}/{load_timestamps[idx]}"
    figure_folder_path = f"{artifacts_folder_path}/{datasets[idx]}/figures/{model_label}/{load_timestamps[idx]}"

    # Create required directories
    os.makedirs(figure_folder_path, exist_ok=True)

    model = NicheCompass.load(dir_path=model_folder_path,
                              adata=None,
                              adata_file_name=f"{datasets[idx]}_{model_label}.h5ad",
                              gp_names_key=gp_names_key)
    
    samples = model.adata.obs[sample_key].unique().tolist()
    
    # Plot mapping entities in latent and physical space
    mapping_entity_colors = create_new_color_dict(
        adata=model.adata,
        skip_default_colors=63,
        cat_key=mapping_entity_key)

    save_fig = True
    file_path = f"{figure_folder_path}/" \
                "mapping_entities_latent_physical_space.svg"

    plot_category_in_latent_and_physical_space(
        adata=model.adata,
        plot_label="Mapping Entities",
        cat_key=mapping_entity_key,
        groups=None,
        sample_key=sample_key,
        samples=samples,
        cat_colors=mapping_entity_colors,
        size=(720000 / len(model.adata)),
        spot_size=spot_size,
        save_fig=save_fig,
        file_path=file_path)

    adata_batch_list = []
    for batch in ["batch1",
                  "batch2",
                  "batch3",
                  "batch4",
                  "batch5",
                  "batch6"]:
        print(f"\nProcessing batch {batch}...")
        print("Loading data...")
        if batch not in ["batch5",
                         "batch6"]:
            adata_batch = ad.read_h5ad(
                f"{st_data_gold_folder_path}/{datasets[idx]}_{batch}.h5ad")
        else:
            adata_batch = ad.read_h5ad(
                f"{st_data_gold_folder_path}/{datasets[0]}_{batch}.h5ad")
        adata_batch_list.append(adata_batch)
    adata_raw = ad.concat(adata_batch_list, join="inner")
    sc.tl.pca(adata_raw, use_highly_variable=False)
    pcr_X_pre = adata_raw.obsm["X_pca"]
    del(adata_raw)

    metrics_dict = compute_benchmarking_metrics(
            adata=model.adata,
            metrics=metrics,
            cell_type_key="celltype_mapped_refined",
            batch_key="batch",
            spatial_key="spatial",
            latent_key="nichecompass_latent",
            pcr_X_pre=pcr_X_pre,
            n_jobs=1,
            seed=0,
            mlflow_experiment_id=None)

    for key, value in metrics_dict.items():
        metrics_dict_all[key].append(value)
    
    model.adata.obs = pd.merge(model.adata.obs, niche_df, left_index=True, right_index=True)
    adata_ref = model.adata[model.adata.obs[mapping_entity_key] == "reference"]
    adata_query = model.adata[model.adata.obs[mapping_entity_key] == "query"]
    
    metrics_dict_all["n_ref_cells"].append(len(adata_ref))

    # Fit KNN classifier for niche label transfer
    knn = KNeighborsClassifier(n_neighbors=8, weights='distance')
    knn.fit(X=adata_ref.obsm['nichecompass_latent'], y=adata_ref.obs["niche"])

    # Transfer niche labels
    proba = knn.predict_proba(adata_query.obsm["nichecompass_latent"])
    k_dist, k_indx = knn.kneighbors(adata_query.obsm["nichecompass_latent"], n_neighbors=8, return_distance=True)
    predictions = proba
    predictions = pd.DataFrame({'predlabel': np.argmax(proba, axis=1), 'probability': np.max(proba, axis=1), 'mean_dist': np.mean(k_dist, axis=1), 'k_dist': k_dist[:,7
    ]})
    predictions['predlabel'] = predictions['predlabel'].map({i: l for i, l in enumerate(knn.classes_)})
    predictions.index = adata_query.obs.index
    predictions
    predictions = pd.merge(predictions, niche_df, left_index=True, right_index=True)
    
    metrics_dict_all["nmi"].append(
        normalized_mutual_info_score(predictions["niche"], predictions["predlabel"]))
    
    adata_query.obs["probability"] = predictions["probability"]
    
    save_fig = True
    file_path = f"{figure_folder_path}/" \
                "niche_label_transfer_proba_latent_physical_space.svg"

    plot_category_in_latent_and_physical_space(
        adata=adata_query,
        plot_label="Label Transfer Probabilities",
        cat_key="probability",
        groups=None,
        sample_key=sample_key,
        samples=samples,
        cat_colors=None,
        size=(720000 / len(adata_query)),
        spot_size=spot_size,
        save_fig=save_fig,
        file_path=file_path)

    os.makedirs(f"{artifacts_folder_path}/{datasets[0]}/results/reference_query", exist_ok=True)
    metrics_df = pd.DataFrame(metrics_dict_all)
    metrics_df.to_csv(f"{artifacts_folder_path}/{datasets[0]}/results/reference_query/power_analysis_metrics.csv", index=False)

In [ ]:
# Plot metrics
melted_metrics_df = metrics_df.melt(
    id_vars="n_ref_cells", value_vars=["blisi", "nmi"],
    var_name="Metric", value_name="value")

melted_metrics_df["Metric"] = melted_metrics_df["Metric"].apply(lambda x: x.replace(
    "blisi", "BLISI").replace(
"nmi", "NMI (Niche"))

melted_metrics_df["n_ref_cells_str"] = melted_metrics_df["n_ref_cells"].apply(lambda x: str(math.ceil(x/100)) + "k")
melted_metrics_df = melted_metrics_df.sort_values(by="n_ref_cells")
plt.figure(figsize=(6, 7))
sns.barplot(x='n_ref_cells_str', y='value', hue='Metric', data=melted_metrics_df, orient='v')
plt.xlabel('Number of Reference Cells', fontsize=16)
plt.ylabel('Metric Value', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(title='Metric', fontsize=18, title_fontsize=18)
plt.savefig(f"{artifacts_folder_path}/{datasets[0]}/results/reference_query/power_analysis_metrics.svg", dpi=300, bbox_inches='tight')
plt.show()

### 2.10 Prior GP Sets

In [ ]:
# Compute metrics
model_label = "reference"
metrics = ["gcs", "mlami", "cas", "clisis", "nasw", "cnmi", "blisi", "pcr"]
    
# Initialize metrics dicts
metrics_dict_all = {"gp_type": []}
metrics_dict_all["n_denovo_gps"] = []
for metric in metrics:
    metrics_dict_all[metric] = []

for idx, gp_type in zip(range(13, 17), ["NicheNet",
                                        "CollecTRI",
                                        "Omnipath",
                                        "MEBOCOST",]):
    model_folder_path = f"{artifacts_folder_path}/{datasets[0]}/models/{model_label}/{load_timestamps[idx]}"
    figure_folder_path = f"{artifacts_folder_path}/{datasets[0]}/figures/{model_label}/{load_timestamps[idx]}"

    # Create required directories
    os.makedirs(figure_folder_path, exist_ok=True)

    model = NicheCompass.load(dir_path=model_folder_path,
                              adata=None,
                              adata_file_name=f"{datasets[0]}_{model_label}.h5ad",
                              gp_names_key=gp_names_key)
    
    metrics_dict_all["gp_type"].append(gp_type)
    metrics_dict_all["n_denovo_gps"].append(model.n_addon_gp_)

    adata_batch_list = []
    for batch in ["batch1",
                  "batch2",
                  "batch3",
                  "batch4",
                  "batch5",
                  "batch6"]:
        print(f"\nProcessing batch {batch}...")
        print("Loading data...")
        if batch not in ["batch5",
                         "batch6"]:
            adata_batch = ad.read_h5ad(
                f"{st_data_gold_folder_path}/{datasets[0]}_{batch}.h5ad")
        else:
            adata_batch = ad.read_h5ad(
                f"{st_data_gold_folder_path}/{datasets[0]}_{batch}.h5ad")
        adata_batch_list.append(adata_batch)
    adata_raw = ad.concat(adata_batch_list, join="inner")
    sc.tl.pca(adata_raw, use_highly_variable=False)
    pcr_X_pre = adata_raw.obsm["X_pca"]
    del(adata_raw)

    metrics_dict = compute_benchmarking_metrics(
            adata=model.adata,
            metrics=metrics,
            cell_type_key="celltype_mapped_refined",
            batch_key="batch",
            spatial_key="spatial",
            latent_key="nichecompass_latent",
            pcr_X_pre=pcr_X_pre,
            n_jobs=1,
            seed=0,
            mlflow_experiment_id=None)

    for key, value in metrics_dict.items():
        metrics_dict_all[key].append(value)
    
    os.makedirs(f"{artifacts_folder_path}/{datasets[0]}/results/reference", exist_ok=True)
    metrics_df = pd.DataFrame(metrics_dict_all)
    metrics_df.to_csv(f"{artifacts_folder_path}/{datasets[0]}/results/reference/prior_gp_analysis_metrics.csv", index=False)

In [ ]:
summary_df

In [ ]:
metric_cols_sample_integration = [
    "cas", "mlami", # global spatial consistency
    "clisis", "gcs", # local spatial consistency
    "nasw", "cnmi", # niche coherence
    "blisi", "pcr" # batch correction
]
metric_col_weights_sample_integration = [ # separate for each category (later multiplied with category_col_weights)
    (1/2), (1/2), # global spatial consistency
    (1/2), (1/2), # local spatial consistency
    1, 1, # niche coherence
    1, 1, # batch correction
]
metric_col_titles_sample_integration = [
    "CAS", # "Cell Type Affinity Similarity",
    "MLAMI", # "Maximum Leiden Adjusted Mutual Info",
    "CLISIS", # "Cell Type Local Inverse Simpson's Index Similarity",
    "GCS", # "Graph Connectivity Similarity",
    "NASW", # "Niche Average Silhouette Width",
    "CNMI", # "Cell Type Normalized Mutual Info",
    "BLISI", # "Batch Local Inverse Simpson's Index"
    "PCR" # "Principal Component Regression"
]
metric_cols_single_sample = metric_cols_sample_integration[:-3]
metric_col_weights_single_sample = metric_col_weights_sample_integration[:-3]
metric_col_titles_single_sample = metric_col_titles_sample_integration[:-3]

category_cols_sample_integration = [
    "Global Spatial Consistency Score",
    "Local Spatial Consistency Score",
    "Niche Coherence Score",
    "Batch Correction Score"]
category_col_weights_sample_integration = [
    1,
    1,
    2,
    2]
category_col_titles_sample_integration = [
    "Global Spatial Consistency Score",
    "Local Spatial Consistency Score",
    "Niche Coherence Score",
    "Batch Correction Score"]
category_col_weights_single_sample = category_col_weights_sample_integration[:-1]
category_cols_single_sample = category_cols_sample_integration[:-1]
category_col_titles_single_sample = [
    "Global Spatial Consistency Score",
    "Local Spatial Consistency Score",
    "Niche Coherence Score"]

# Visualize metrics
summary_df = pd.read_csv(f"{artifacts_folder_path}/seqfish_mouse_organogenesis_imputed/results/reference/prior_gp_analysis_metrics.csv")

# Apply min-max scaling to metric columns
for i in range(len(metric_cols_sample_integration)):
    min_val = summary_df[metric_cols_sample_integration[i]].min()
    max_val = summary_df[metric_cols_sample_integration[i]].max()
    summary_df[metric_cols_sample_integration[i] + "_scaled"] = ((
        summary_df[metric_cols_sample_integration[i]] - min_val) / (max_val - min_val))
    
cat_0_scaled_metric_cols = [metric_col + "_scaled" for metric_col in metric_cols_sample_integration[0:2]]
cat_1_scaled_metric_cols = [metric_col + "_scaled" for metric_col in metric_cols_sample_integration[2:4]]
cat_2_scaled_metric_cols = [metric_col + "_scaled" for metric_col in metric_cols_sample_integration[4:6]]
cat_3_scaled_metric_cols = [metric_col + "_scaled" for metric_col in metric_cols_sample_integration[6:8]]
    
summary_df[category_cols_sample_integration[0]] = np.average(summary_df[cat_0_scaled_metric_cols],
                                                        weights=metric_col_weights_sample_integration[0:2],
                                                        axis=1)
summary_df[category_cols_sample_integration[1]] = np.average(summary_df[cat_1_scaled_metric_cols],
                                                        weights=metric_col_weights_sample_integration[2:4],
                                                        axis=1)
summary_df[category_cols_sample_integration[2]] = np.average(summary_df[cat_2_scaled_metric_cols],
                                                        weights=metric_col_weights_sample_integration[4:6],
                                                        axis=1)
summary_df[category_cols_sample_integration[3]] = np.average(summary_df[cat_3_scaled_metric_cols],
                                                        weights=metric_col_weights_sample_integration[6:8],
                                                        axis=1)
summary_df["Overall Score"] = np.average(summary_df[category_cols_sample_integration[:4]],
                                         weights=category_col_weights_sample_integration[:4],
                                         axis=1)

group_cols = ["gp_type"]

summary_df = summary_df[group_cols +
                        metric_cols_sample_integration +
                        ["Overall Score"]].reset_index()

mapping_dict = {
"Omnipath": "Ligand-Receptor GPs",
"MEBOCOST": "Metabolite-Sensor GPs",
"CollecTRI": "Transcriptional Regulation GPs",
"NicheNet": "Combined Interaction GPs"}

summary_df["GP Set"] = summary_df["gp_type"].apply(
    lambda x: mapping_dict.get(x))

custom_order = ["Ligand-Receptor GPs",
                "Metabolite-Sensor GPs",
                "Transcriptional Regulation GPs",
                "Combined Interaction GPs"]

summary_df["GP Set"] = pd.Categorical(summary_df["GP Set"], categories=custom_order, ordered=True)
summary_df = summary_df.sort_values(by='GP Set')

for metric in metrics:
    plt.figure(figsize=(5, 2))
    sns.barplot(x=metric, y='GP Set', data=summary_df, orient='h')
    plt.xlabel(metric.upper(), fontsize=16)
    plt.ylabel("GP Set", fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.savefig(f"{artifacts_folder_path}/{datasets[0]}/results/reference/{metric}.svg", dpi=300, bbox_inches='tight')
    plt.show()

#### 2.10.1 NicheNet

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{datasets[0]}/models/{model_label}/{load_timestamps[13]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[0]}/figures/{model_label}/{load_timestamps[13]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[0]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.38
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "6"
midbrain_cluster = "3"
hindbrain_cluster = "7"
floorplate_cluster = "17"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "2"
dorsalgut_cluster = "18"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1

In [ ]:
model.adata.uns[differential_gp_test_results_key] = model.adata.uns[differential_gp_test_results_key][
    model.adata.uns[differential_gp_test_results_key]["category"].isin(
        [#forebrain_cluster,
         midbrain_cluster,
         #hindbrain_cluster,
         #floorplate_cluster,
         #ventralgut_cluster,
         #dorsalgut_cluster
        ])]

In [ ]:
save_figs = True

generate_enriched_gp_info_plots(
    plot_label="Top enriched GPs and GP genes",
    model=model,
    sample_key=sample_key,
    differential_gp_test_results_key=differential_gp_test_results_key,
    cat_key=latent_cluster_key,
    cat_palette=latent_cluster_colors,
    n_top_enriched_gp_start_idx=0,
    n_top_enriched_gp_end_idx=10,
    feature_spaces=["embryo2"],
    n_top_genes_per_gp=3,
    save_figs=save_figs,
    figure_folder_path=f"{figure_folder_path}/",
    spot_size=spot_size)

In [ ]:
gp = "Trh_ligand_receptor_target_gene_GP"
gene = "Trh"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo2"],
    color=gp,
    color_map="RdGy_r",
    spot_size=spot_size,
    title=gp,
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo2"],
    color=gene,
    color_map="RdPu",
    spot_size=spot_size,
    title=gp,
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/characterizing_gp_gene.svg",
            bbox_inches="tight")

#### 2.10.2 CollecTRI

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{datasets[0]}/models/{model_label}/{load_timestamps[14]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[0]}/figures/{model_label}/{load_timestamps[14]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[0]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.3
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "1"
midbrain_cluster = "5"
hindbrain_cluster = "10"
floorplate_cluster = "15"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "4"
dorsalgut_cluster = "14"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1

In [ ]:
model.adata.uns[differential_gp_test_results_key] = model.adata.uns[differential_gp_test_results_key][
    model.adata.uns[differential_gp_test_results_key]["category"].isin(
        [#forebrain_cluster,
         midbrain_cluster,
         #hindbrain_cluster,
         #floorplate_cluster,
         #ventralgut_cluster,
         #dorsalgut_cluster
        ])]

In [ ]:
save_figs = True

generate_enriched_gp_info_plots(
    plot_label="Top enriched GPs and GP genes",
    model=model,
    sample_key=sample_key,
    differential_gp_test_results_key=differential_gp_test_results_key,
    cat_key=latent_cluster_key,
    cat_palette=latent_cluster_colors,
    n_top_enriched_gp_start_idx=0,
    n_top_enriched_gp_end_idx=10,
    feature_spaces=["embryo2"],
    n_top_genes_per_gp=3,
    save_figs=save_figs,
    figure_folder_path=f"{figure_folder_path}/",
    spot_size=spot_size)

In [ ]:
gp = "Sox21_TF_target_genes_GP"
gene = "Sox21"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo2"],
    color=gp,
    color_map="RdGy_r",
    spot_size=spot_size,
    title=gp,
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo2"],
    color=gene,
    color_map="RdPu",
    spot_size=spot_size,
    title=gp,
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/characterizing_gp_gene.svg",
            bbox_inches="tight")

#### 2.10.3 Omnipath

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{datasets[0]}/models/{model_label}/{load_timestamps[15]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[0]}/figures/{model_label}/{load_timestamps[15]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[0]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.25
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "0"
midbrain_cluster = "13"
hindbrain_cluster = "5"
floorplate_cluster = "15"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "9"
dorsalgut_cluster = "24"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1

In [ ]:
model.adata.uns[differential_gp_test_results_key] = model.adata.uns[differential_gp_test_results_key][
    model.adata.uns[differential_gp_test_results_key]["category"].isin(
        [#forebrain_cluster,
         midbrain_cluster,
         #hindbrain_cluster,
         #floorplate_cluster,
         #ventralgut_cluster,
         #dorsalgut_cluster
        ])]

In [ ]:
save_figs = True

generate_enriched_gp_info_plots(
    plot_label="Top enriched GPs and GP genes",
    model=model,
    sample_key=sample_key,
    differential_gp_test_results_key=differential_gp_test_results_key,
    cat_key=latent_cluster_key,
    cat_palette=latent_cluster_colors,
    n_top_enriched_gp_start_idx=0,
    n_top_enriched_gp_end_idx=10,
    feature_spaces=["embryo2"],
    n_top_genes_per_gp=3,
    save_figs=save_figs,
    figure_folder_path=f"{figure_folder_path}/",
    spot_size=spot_size)

In [ ]:
gp = "WNT1_ligand_receptor_GP"
gene = "Wnt1"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo2"],
    color=gp,
    color_map="RdGy_r",
    spot_size=spot_size,
    title=gp,
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo2"],
    color=gene,
    color_map="RdPu",
    spot_size=spot_size,
    title=gp,
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/characterizing_gp_gene.svg",
            bbox_inches="tight")

#### 2.10.4 MEBOCOST

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{datasets[0]}/models/{model_label}/{load_timestamps[16]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[0]}/figures/{model_label}/{load_timestamps[16]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[0]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.35
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "0"
midbrain_cluster = "1"
hindbrain_cluster = "10"
floorplate_cluster = "17"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "6"
dorsalgut_cluster = "22"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1

In [ ]:
model.adata.uns[differential_gp_test_results_key] = model.adata.uns[differential_gp_test_results_key][
    model.adata.uns[differential_gp_test_results_key]["category"].isin(
        [#forebrain_cluster,
         midbrain_cluster,
         #hindbrain_cluster,
         #floorplate_cluster,
         #ventralgut_cluster,
         #dorsalgut_cluster
        ])]

In [ ]:
save_figs = True

generate_enriched_gp_info_plots(
    plot_label="Top enriched GPs and GP genes",
    model=model,
    sample_key=sample_key,
    differential_gp_test_results_key=differential_gp_test_results_key,
    cat_key=latent_cluster_key,
    cat_palette=latent_cluster_colors,
    n_top_enriched_gp_start_idx=0,
    n_top_enriched_gp_end_idx=10,
    feature_spaces=["embryo2"],
    n_top_genes_per_gp=3,
    save_figs=save_figs,
    figure_folder_path=f"{figure_folder_path}/",
    spot_size=spot_size)

In [ ]:
gp = "Palmitic acid_metabolite_enzyme_sensor_GP"
gene = "Acot1"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo2"],
    color=gp,
    color_map="RdGy_r",
    spot_size=spot_size,
    title=gp,
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata[model.adata.obs["sample"] == "embryo2"],
    color=gene,
    color_map="RdPu",
    spot_size=spot_size,
    title=gp,
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/characterizing_gp_gene.svg",
            bbox_inches="tight")

### 2.11 Radius-based Neighborhood Graph

In [ ]:
# Determine radius to obtain an average number of neighbors of 8
adata_batch = sc.read_h5ad(f"{st_data_gold_folder_path}/seqfish_mouse_organogenesis_imputed_batch1.h5ad")

In [ ]:
mean_n_neighs_knn = []
mean_n_neighs_radius = []

for batch_idx in range(1,6):
    adata_batch = sc.read_h5ad(f"{st_data_gold_folder_path}/seqfish_mouse_organogenesis_imputed_batch{batch_idx}.h5ad")
    
    sq.gr.spatial_neighbors(adata_batch,
                            coord_type="generic",
                            spatial_key=spatial_key,
                            n_neighs=8)

    # Make adjacency matrix symmetric
    adata_batch.obsp[adj_key] = (
        adata_batch.obsp[adj_key].maximum(
            adata_batch.obsp[adj_key].T))
    
    mean_n_neighs_knn.append(np.mean(adata_batch.obsp["spatial_connectivities"].sum(1)))

    sq.gr.spatial_neighbors(adata_batch,
                            coord_type="generic",
                            spatial_key=spatial_key,
                            radius=0.08)
    
    mean_n_neighs_radius.append(np.mean(adata_batch.obsp["spatial_connectivities"].sum(1)))
    
print(np.mean(mean_n_neighs_knn))
print(np.mean(mean_n_neighs_radius))

In [ ]:
model_label = "reference"
model_folder_path = f"{artifacts_folder_path}/{datasets[0]}/models/{model_label}/{load_timestamps[17]}"
figure_folder_path = f"{artifacts_folder_path}/{datasets[0]}/figures/{model_label}/{load_timestamps[17]}"


# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)

model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{datasets[4]}_{model_label}.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
samples = model.adata.obs[sample_key].unique().tolist()

In [ ]:
# Identify and plot niches
latent_leiden_resolution = 0.35
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)

sc.tl.leiden(adata=model.adata,
             resolution=0.05,
             key_added=latent_cluster_key,
             restrict_to=(latent_cluster_key, ["0"]),
             neighbors_key=latent_key)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    color_palette="default",
    cat_key=latent_cluster_key)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "latent_clusters_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot CNS niches
forebrain_cluster = "4"
midbrain_cluster = "3"
hindbrain_cluster = "10"
floorplate_cluster = "14"

latent_cluster_colors[forebrain_cluster] = "#B497E7"
latent_cluster_colors[midbrain_cluster] = "#FF69B4"
latent_cluster_colors[hindbrain_cluster] = "#276A8C"
latent_cluster_colors[floorplate_cluster] = "#C38D9E"

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "forebrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[forebrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "midbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[midbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "hindbrain_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[hindbrain_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "floorplate_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[floorplate_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Plot gut niches
ventralgut_cluster = "6"
dorsalgut_cluster = "20"

latent_cluster_colors[ventralgut_cluster] = "#F6CF71"
latent_cluster_colors[dorsalgut_cluster] = "#9D88A2"

# Ventral Gut
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "ventralgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[ventralgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "dorsalgut_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key=latent_cluster_key,
    groups=[dorsalgut_cluster],
    sample_key=sample_key,
    samples=samples,
    cat_colors=latent_cluster_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
# Run differential GP testing
selected_cats = None
log_bayes_factor_thresh = 2.3

enriched_gps = model.run_differential_gp_tests(
    cat_key=latent_cluster_key,
    selected_cats=selected_cats,
    comparison_cats="rest",
    log_bayes_factor_thresh=log_bayes_factor_thresh)

# Check whether GPs are enriched
niche_gps = {}
niche_gps[forebrain_cluster] = ["PCSK1N_ligand_receptor_GP", "DKK1_ligand_receptor_GP"]
niche_gps[midbrain_cluster] = ["Fgf17_ligand_receptor_target_gene_GP", "Efna2_ligand_receptor_target_gene_GP"]
niche_gps[hindbrain_cluster] = ["Gdf10_ligand_receptor_target_gene_GP", "Fgf3_ligand_receptor_target_gene_GP"]
niche_gps[floorplate_cluster] = ["CALCA_ligand_receptor_GP", "Shh_ligand_receptor_target_gene_GP"]
niche_gps[ventralgut_cluster] = ["IHH_ligand_receptor_GP", "Spint1_ligand_receptor_target_gene_GP"]
niche_gps[dorsalgut_cluster] = ["Cthrc1_ligand_receptor_target_gene_GP", "PDGFC_ligand_receptor_GP"]

overlap_gps = {}
for niche, gps in niche_gps.items():
    niche_enriched_gps = model.adata.uns[differential_gp_test_results_key][
        model.adata.uns[differential_gp_test_results_key]["category"] == niche]["gene_program"].values.tolist()
    overlap_gps[niche] = [gp for gp in gps if gp in niche_enriched_gps]

# Correct sign of GPs
gp_summary_df = model.get_gp_summary()
model.add_active_gp_scores_to_obs()

for idx, row in gp_summary_df[gp_summary_df["gp_active"]].iterrows():
    if np.array(row["gp_source_genes_weights"]).sum() < 0:
        model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
    elif np.array(row["gp_source_genes_weights"]).sum() == 0:
        if np.array(row["gp_target_genes_weights"]).sum() < 0:
            model.adata.obs[row["gp_name"]] = model.adata.obs[row["gp_name"]] * -1
            
all_gps = []
for sublist in list(niche_gps.values()):
    all_gps.extend(sublist)

In [ ]:
# Plot CNS GPs
adata_top = model.adata[model.adata.obsm["spatial"][:,1] < 0]
adata_top = adata_top[adata_top.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_top.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_top.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_top.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=4,
                        ncols=2,
                        figsize=(15, 15))

# Forebrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pcsk1n GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[forebrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Dkk1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Midbrain
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf17 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[midbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Efna2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)

# Floor plate
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][0]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Calca GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[floorplate_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Shh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)

# Hindbrain
#sc.pl.spatial(
#    adata=adata_top,
#    color=f"{niche_gps[hindbrain_cluster][0]}_normalized",
#    color_map=color_map,
#    spot_size=spot_size,
#    title=f"Gdf10 GP",
#    legend_loc=None,
#    colorbar_loc="bottom",
#    ax=axs[3, 0],
#    show=False) # inactive GP
sc.pl.spatial(
    adata=adata_top,
    color=f"{niche_gps[hindbrain_cluster][1]}_normalized",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fgf3 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/brain_enriched_gps_top.svg",
            bbox_inches="tight")

In [ ]:
# Plot gut GPs
adata_gut = model.adata[model.adata.obsm["spatial"][:,1] < 2.8]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,1] > -1.6]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] < 1.3]
adata_gut = adata_gut[adata_gut.obsm["spatial"][:,0] > -1.2]
adata_gut = adata_gut[adata_gut.obs["sample"] == "embryo2"]

for gp in all_gps:
    if gp in adata_gut.obs.columns:
        scaler = MinMaxScaler()
        # Fit and transform the column to normalize its values
        adata_gut.obs[f'{gp}_normalized'] = scaler.fit_transform(adata_gut.obs[[gp]])

color_map = "RdGy_r"
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Ventral Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][0]}", # not normalized (to match original manuscript)
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cthrc1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[dorsalgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Pdgfc GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)

# Dorsal Gut
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][0]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ihh GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=adata_gut,
    color=f"{niche_gps[ventralgut_cluster][1]}",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Spint1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/gut_enriched_gps.svg",
            bbox_inches="tight")